In [58]:
url = "https://www.espncricinfo.com/ci/content/player/index.html?country=6"

In [59]:
from bs4 import BeautifulSoup as bf
import requests
import pandas as pd

In [60]:
req = requests.get(url)

In [61]:
soup = bf(req.text,"html.parser")

In [62]:
jo = soup.find_all('td')

In [63]:
#Fetch total urls of players belonging to a country
u = list()
for job_elem in jo:
    u.append(job_elem.find('a'))
    #result.append(job_elem.text)
l = set(u)
l = list(l)

In [108]:
#Get relevant information related to a player using each player's url
def load(req,lol,ind,u):
    lab = list()
    result = list()
    soup = bf(req.text,"html.parser")
    print(soup.title)
    jo = soup.find_all('h5',class_='player-card-description gray-900')
    c = 0
    for job_elem in jo:
        result.append(job_elem.text)
        c+=1
    jo = soup.find_all('p',class_="text-uppercase gray-700 mb-0 pb-0-5 player-card-heading")
    for job_elem in jo:
        lab.append(job_elem.text)
    ######################################
    rec = extra(req,u)
    lab.append("Records")
    result.append([rec])
    print(result)
    ##########################################   
    jo = soup.find('div',class_="player-card__details")
    l = list()
    for job_elem in jo:
        l.append(job_elem.text)
        c+=1
    p = l[1].split("|")
    l.pop()
    l.pop()
    l = l+p
    result = result + l
    l_label = ["Player Name" , "Nationality ", "player-type"]
    lab = lab + l_label
    ##########################
    jo = soup.body.findAll('thead')
    kl = jo[0].find_all('th')
    head = 0
    hd_el = list()
    hd_l =list()
    for job_elem in kl:
        hd_el.append(job_elem.text)
        head+=1
    print('Head count',head)
    
    jo = soup.body.findAll('tbody')
    kl = jo[0].find_all('td')
    t = 0
    for job_elem in kl:
        if(t%head !=0):
            result.append(job_elem.text)
            c+=1
        else:
            hd_l.append(job_elem.text)
        
        t+=1
    total_label1 = list()
    for i in range(1,len(hd_el)):
        for j in range(len(hd_l)):
            total_label1.append('Batting' + hd_el[i] +" "+hd_l[j])
    lab = lab + total_label1
    ####################################
    jo = soup.body.findAll('thead')
    kl = jo[1].find_all('th')
    head = 0
    hd_el = list()
    hd_l =list()
    for job_elem in kl:
        hd_el.append(job_elem.text)
        head+=1
    print('Head count',head)
    
    jo = soup.body.findAll('tbody')
    kl = jo[1].find_all('td')
    t = 0
    for job_elem in kl:
        if(t%head !=0):
            result.append(job_elem.text)
            c+=1
        else:
            hd_l.append(job_elem.text)
        
        t+=1
    total_label1 = list()
    for i in range(1,len(hd_el)):
        for j in range(len(hd_l)):
            total_label1.append('Bowling' + hd_el[i] +" "+hd_l[j])
    lab = lab + total_label1
    print('label--',len(lab))
    print('result--',len(result))
    o = dict()
    for i in range(len(lab)):
        o[lab[i]] = result[i]
    #print(lab)
    
    lo = dict(zip(lab,result))
    lp = pd.DataFrame(lo,index=[ind])
    lol.append(lp)
    print('count',c)

In [109]:
#Get individual player's "Records" information.
import re
def extra(req,l):
    soup = bf(req.text,"html.parser")
    print(req.url)
    #print(soup.title)
    jo = soup.find_all('a',class_='more_content-footer')
    for i in range(len(jo)):
        p = jo[i]['href']
        if('t20-records' in p):
            #print(l + p)
            u = l+p
            re = requests.get(u)
            so = bf(re.text,"html.parser")
            res1 = so.find_all('div',class_='col-14 d-flex flex-row')
            res = so.find_all('div',class_='row bg-gray-100 class-record-row-p d-flex align-items-center')
            #print(res1)
            url_list = list()
            for i in range(len(res1)):
                
                pre = res1[i].find_all('h1')
                loi = res1[i].find_all('p',class_='record-class-title m-0')
                #print(loi)    
                for j in range(len(pre)):
                    #print(pre[j].text)
                    #print(loi[j].text)
                    url_list.append(str(pre[j].text)  +str(" ") + str(loi[j].text))
            return url_list


In [73]:
#extra code
            """
            for i in range(len(res)):
                la = res[i].find_all('span')
                #print(la)
                for j in range(len(la)):
                    oi = la[j].text
                    print(oi,end=" ")
                print()
            """
            #print(la)
    print("-------------------------------------------")
    #print(l + jo['href'])

IndentationError: ignored

In [111]:
lol = list()
for i in range(1,10):
    u = "https://www.espncricinfo.com"
    rl = l[i]["href"]
    link = u+rl
    #print(link)
    req = requests.get(link)
    print('player-',i,end=" ")
    ind = i-1
    load(req,lol,ind,u)
    #jk = extra(req,u)
    #print(jk)

player- 1 <title>Navdeep Saini profile and biography, stats, records, averages, photos and videos</title>
https://www.espncricinfo.com/player/navdeep-saini-700167
['Navdeep Amarjeet Saini', 'November 23, 1992, Karnal, Haryana', '28y 182d', 'Right hand bat', 'Right arm fast', 'Bowler', [None]]
Head count 14
Head count 15
label-- 172
result-- 172
count 171
player- 2 <title>Ravichandran Ashwin profile and biography, stats, records, averages, photos and videos</title>
https://www.espncricinfo.com/player/ravichandran-ashwin-26421
['Ravichandran Ashwin', 'September 17, 1986, Madras (now Chennai), Tamil Nadu', '34y 249d', 'Right hand bat', 'Right arm offbreak', 'Bowling allrounder', [['3rd Most player-of-the-series awards (8)', '6th Most ten-wickets-in-a-match in a career (7)', '1st Fastest to 350 wickets (66)', ' A hundred and five wickets in an innings ', '9th Most runs in an innings (by batting position) (31*)', '9th Outstanding bowling analyses in an innings (4/8)', '3rd Bowler/fielder co

In [112]:
a = lol[0]
b = lol[1]

In [97]:
oi = {'Full Name': 'Prithvi Pankaj Shaw', 'Born': 'November 09, 1999, Thane, Maharashtra', 'Age': '21y 196d', 'Batting Style': 'Right hand bat', 'Bowling Style': 'Right arm offbreak', 'Playing Role': 'Opening batter', 'Records': [' Hundred on debut (134)', '7th Youngest player to score a hundred (18y 329d)']}
lp = pd.DataFrame(oi,ignore_index=False)

TypeError: ignored

In [100]:
io = { 'Records': [[' Hundred on debut (134)', '7th Youngest player to score a hundred (18y 329d)']]}
pd.DataFrame(io , index=[0])

,Records
0,"[ Hundred on debut (134), 7th Youngest player ..."


In [104]:
p = [1,2]

In [107]:
[None]

[None]

In [113]:
df = a.T
for i in range(1,len(lol)):
    lo = lol[i].T
    df = pd.concat([df,lo],axis=1)

In [ ]:
df5 = pd.concat([o,r,c],axis=1)

In [118]:
df.T

,Full Name,Born,Age,Batting Style,Bowling Style,Playing Role,Records,Player Name,Nationality,player-type,BattingMat Test,BattingMat ODI,BattingMat T20I,BattingMat FC,BattingMat List A,BattingMat T20,BattingInns Test,BattingInns ODI,BattingInns T20I,BattingInns FC,BattingInns List A,BattingInns T20,BattingBalls Test,BattingBalls ODI,BattingBalls T20I,BattingBalls FC,BattingBalls List A,BattingBalls T20,BattingRuns Test,BattingRuns ODI,BattingRuns T20I,BattingRuns FC,BattingRuns List A,BattingRuns T20,BattingWkts Test,BattingWkts ODI,BattingWkts T20I,BattingWkts FC,BattingWkts List A,BattingWkts T20,BattingBBI Test,BattingBBI ODI,BattingBBI T20I,BattingBBI FC,BattingBBI List A,BattingBBI T20,BattingBBM Test,BattingBBM ODI,BattingBBM T20I,BattingBBM FC,BattingBBM List A,BattingBBM T20,BattingAve Test,BattingAve ODI,BattingAve T20I,BattingAve FC,BattingAve List A,BattingAve T20,BattingEcon Test,BattingEcon ODI,BattingEcon T20I,BattingEcon FC,BattingEcon List A,BattingEcon T20,BattingSR Test,BattingSR ODI,BattingSR T20I,BattingSR FC,BattingSR List A,BattingSR T20,Batting4w Test,Batting4w ODI,Batting4w T20I,Batting4w FC,Batting4w List A,Batting4w T20,Batting5w Test,Batting5w ODI,Batting5w T20I,Batting5w FC,Batting5w List A,Batting5w T20,Batting10w Test,Batting10w ODI,Batting10w T20I,Batting10w FC,Batting10w List A,Batting10w T20,BowlingMat Test,BowlingMat ODI,BowlingMat T20I,BowlingMat FC,BowlingMat List A,BowlingMat T20,BowlingInns Test,BowlingInns ODI,BowlingInns T20I,BowlingInns FC,BowlingInns List A,BowlingInns T20,BowlingNO Test,BowlingNO ODI,BowlingNO T20I,BowlingNO FC,BowlingNO List A,BowlingNO T20,BowlingRuns Test,BowlingRuns ODI,BowlingRuns T20I,BowlingRuns FC,BowlingRuns List A,BowlingRuns T20,BowlingHS Test,BowlingHS ODI,BowlingHS T20I,BowlingHS FC,BowlingHS List A,BowlingHS T20,BowlingAve Test,BowlingAve ODI,BowlingAve T20I,BowlingAve FC,BowlingAve List A,BowlingAve T20,BowlingBF Test,BowlingBF ODI,BowlingBF T20I,BowlingBF FC,BowlingBF List A,BowlingBF T20,BowlingSR Test,BowlingSR ODI,BowlingSR T20I,BowlingSR FC,BowlingSR List A,BowlingSR T20,Bowling100s Test,Bowling100s ODI,Bowling100s T20I,Bowling100s FC,Bowling100s List A,Bowling100s T20,Bowling50s Test,Bowling50s ODI,Bowling50s T20I,Bowling50s FC,Bowling50s List A,Bowling50s T20,Bowling4s Test,Bowling4s ODI,Bowling4s T20I,Bowling4s FC,Bowling4s List A,Bowling4s T20,Bowling6s Test,Bowling6s ODI,Bowling6s T20I,Bowling6s FC,Bowling6s List A,Bowling6s T20,BowlingCt Test,BowlingCt ODI,BowlingCt T20I,BowlingCt FC,BowlingCt List A,BowlingCt T20,BowlingSt Test,BowlingSt ODI,BowlingSt T20I,BowlingSt FC,BowlingSt List A,BowlingSt T20,BattingNO ODI,BattingNO T20I,BattingNO FC,BattingNO List A,BattingNO T20,BattingHS ODI,BattingHS T20I,BattingHS FC,BattingHS List A,BattingHS T20,BattingBF ODI,BattingBF T20I,BattingBF FC,BattingBF List A,BattingBF T20,Batting100s ODI,Batting100s T20I,Batting100s FC,Batting100s List A,Batting100s T20,Batting50s ODI,Batting50s T20I,Batting50s FC,Batting50s List A,Batting50s T20,Batting4s ODI,Batting4s T20I,Batting4s FC,Batting4s List A,Batting4s T20,Batting6s ODI,Batting6s T20I,Batting6s FC,Batting6s List A,Batting6s T20,BattingCt ODI,BattingCt T20I,BattingCt FC,BattingCt List A,BattingCt T20,BattingSt ODI,BattingSt T20I,BattingSt FC,BattingSt List A,BattingSt T20,BowlingBalls ODI,BowlingBalls T20I,BowlingBalls FC,BowlingBalls List A,BowlingBalls T20,BowlingWkts ODI,BowlingWkts T20I,BowlingWkts FC,BowlingWkts List A,BowlingWkts T20,BowlingBBI ODI,BowlingBBI T20I,BowlingBBI FC,BowlingBBI List A,BowlingBBI T20,BowlingBBM ODI,BowlingBBM T20I,BowlingBBM FC,BowlingBBM List A,BowlingBBM T20,BowlingEcon ODI,BowlingEcon T20I,BowlingEcon FC,BowlingEcon List A,BowlingEcon T20,Bowling4w ODI,Bowling4w T20I,Bowling4w FC,Bowling4w List A,Bowling4w T20,Bowling5w ODI,Bowling5w T20I,Bowling5w FC,Bowling5w List A,Bowling5w T20,Bowling10w ODI,Bowling10w T20I,Bowling10w FC,Bowling10w List A,Bowling10w T20,BattingNO Test,BattingHS Test,BattingBF Test,Batting100s Test,B

In [ ]:
c =lol[2].T

In [116]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
    ################################################3
    jo = soup.body.findAll('thead')
    kl = jo[0].find_all('th')
    head = 0
    hd_el = list()
    hd_l =list()
    for job_elem in kl:
        hd_el.append(job_elem.text)
        head+=1
    print('Head count',head)
    
    jo = soup.body.findAll('tbody')
    kl = jo[0].find_all('td')
    t = 0
    for job_elem in kl:
        if(t%head !=0):
            result.append(job_elem.text)
            c+=1
        else:
            hd_l.append(job_elem.text)
        
        t+=1
    total_label1 = list()
    for i in range(len(hd_el)):
        for j in range(len(hd_l)):
            total_label1.append(hd_el[i] +" "+hd_l[j])
    ####################################################
    jo = soup.body.findAll('thead')
    kl = jo[1].find_all('th')
    head1 = 0
    hd_el1 = list()
    hd1_l = list()
    for job_elem in kl:
        hd_el1.append(job_elem.text)
        head1+=1
    print('Head1 count',head1)
    
    kl = jo[1].find_all('td')
    t = 0
    for job_elem in kl:
        if(t%head1 !=0):
            result.append(job_elem.text)
            c+=1
        else:
            hd1_l.append(job_elem.text)
        
        t+=1
    total_label2 = list()
    for i in range(len(hd_el1)):
        for j in range(len(hd1_l)):
            total_label2.append(hd_el1[i] +" "+hd1_l[j])
    lab = lab + total_label1 + total_label2
    print(len(lab),end="---")
    print(len(result))
    print('count',c)

IndexError: list index out of range

In [ ]:
for i in range(len(l)):
    u = "https://www.espncricinfo.com"
    rl = l[i]["href"]
    link = u+rl
    #print(link)
    req = requests.get(link)
    print('player-',i,end=" ")
    load(req)

player- 0 count 184
player- 1 count 184
player- 2 count 183
player- 3 count 156
player- 4 count 184
player- 5 count 125
player- 6 count 126
player- 7 count 183
player- 8 count 155
player- 9 count 184
player- 10 count 125
player- 11 count 182
player- 12 count 155
player- 13 count 153
player- 14 count 125
player- 15 count 126
player- 16 count 183
player- 17 count 184
player- 18 count 156
player- 19 count 125
player- 20 count 183
player- 21 count 126
player- 22 count 183
player- 23 count 184
player- 24 count 183
player- 25 count 154
player- 26 count 185
player- 27 count 184
player- 28 count 125
player- 29 count 183
player- 30 count 184
player- 31 count 183
player- 32 count 183
player- 33 count 184
player- 34 count 183
player- 35 count 185
player- 36 count 125
player- 37 count 154
player- 38 count 183
player- 39 count 184
player- 40 count 184
player- 41 count 184


In [ ]:
l[0]

<a href="/ci/content/player/373538.html">Sabbir Rahman</a>

In [ ]:
ls

cricket_data.csv      full_dataset.ipynb  new.ipynb           Untitled1.ipynb
cricket_data.csv.zip  l.py                player_dataset.csv  Untitled.ipynb


In [ ]:
p = pd.read_csv('cricket_data.csv')

/home/hrudai/anaconda3/envs/indic/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (9,16,20,21,22,24,25,26,27,28,29,30,31,32,45,46,59,60,73,74,87,88,90,91,92,95,96,97,98,99,100,101,102) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
p

,ID,NAME,COUNTRY,Full name,Born,Died,Current age,Major teams,Education,Height,Nickname,Playing role,Batting style,Bowling style,Other,Relation,In a nutshell,DESCRIPTION,AWARDS,BATTING_Tests_Mat,BATTING_Tests_Inns,BATTING_Tests_NO,BATTING_Tests_Runs,BATTING_Tests_HS,BATTING_Tests_Ave,BATTING_Tests_BF,BATTING_Tests_SR,BATTING_Tests_100,BATTING_Tests_50,BATTING_Tests_4s,BATTING_Tests_6s,BATTING_Tests_Ct,BATTING_Tests_St,BATTING_ODIs_Mat,BATTING_ODIs_Inns,BATTING_ODIs_NO,BATTING_ODIs_Runs,BATTING_ODIs_HS,BATTING_ODIs_Ave,BATTING_ODIs_BF,BATTING_ODIs_SR,BATTING_ODIs_100,BATTING_ODIs_50,BATTING_ODIs_4s,BATTING_ODIs_6s,BATTING_ODIs_Ct,BATTING_ODIs_St,BATTING_T20Is_Mat,BATTING_T20Is_Inns,BATTING_T20Is_NO,BATTING_T20Is_Runs,BATTING_T20Is_HS,BATTING_T20Is_Ave,BATTING_T20Is_BF,BATTING_T20Is_SR,BATTING_T20Is_100,BATTING_T20Is_50,BATTING_T20Is_4s,BATTING_T20Is_6s,BATTING_T20Is_Ct,BATTING_T20Is_St,BATTING_First-class_Mat,BATTING_First-class_Inns,BATTING_First-class_NO,BATTING_First-class_Runs,BATTING_First-class_HS,BATTING_First-class_Ave,BATTING_First-class_BF,BATTING_First-class_SR,BATTING_First-class_100,BATTING_First-class_50,BATTING_First-class_4s,BATTING_First-class_6s,BATTING_First-class_Ct,BATTING_First-class_St,BATTING_List A_Mat,BATTING_List A_Inns,BATTING_List A_NO,BATTING_List A_Runs,BATTING_List A_HS,BATTING_List A_Ave,BATTING_List A_BF,BATTING_List A_SR,BATTING_List A_100,BATTING_List A_50,BATTING_List A_4s,BATTING_List A_6s,BATTING_List A_Ct,BATTING_List A_St,BATTING_T20s_Mat,BATTING_T20s_Inns,BATTING_T20s_NO,BATTING_T20s_Runs,BATTING_T20s_HS,BATTING_T20s_Ave,BATTING_T20s_BF,BATTING_T20s_SR,BATTING_T20s_100,BATTING_T20s_50,BATTING_T20s_4s,BATTING_T20s_6s,BATTING_T20s_Ct,BATTING_T20s_St,BOWLING_Tests_Mat,BOWLING_Tests_Inns,BOWLING_Tests_Balls,BOWLING_Tests_Runs,BOWLING_Tests_Wkts,BOWLING_Tests_BBI,BOWLING_Tests_BBM,BOWLING_Tests_Ave,BOWLING_Tests_Econ,BOWLING_Tests_SR,BOWLING_Tests_4w,BOWLING_Tests_5w,BOWLING_Tests_10,BOWLING_ODIs_Mat,BOWLING_ODIs_Inns,BOWLING_ODIs_Balls,BOWLING_ODIs_Runs,BOWLING_ODIs_Wkts,BOWLING_ODIs_BBI,BOWLING_ODIs_BBM,BOWLING_ODIs_Ave,BOWLING_ODIs_Econ,BOWLING_ODIs_SR,BOWLING_ODIs_4w,BOWLING_ODIs_5w,BOWLING_ODIs_10,BOWLING_T20Is_Mat,BOWLING_T20Is_Inns,BOWLING_T20Is_Balls,BOWLING_T20Is_Runs,BOWLING_T20Is_Wkts,BOWLING_T20Is_BBI,BOWLING_T20Is_BBM,BOWLING_T20Is_Ave,BOWLING_T20Is_Econ,BOWLING_T20Is_SR,BOWLING_T20Is_4w,BOWLING_T20Is_5w,BOWLING_T20Is_10,BOWLING_First-class_Mat,BOWLING_First-class_Inns,BOWLING_First-class_Balls,BOWLING_First-class_Runs,BOWLING_First-class_Wkts,BOWLING_First-class_BBI,BOWLING_First-class_BBM,BOWLING_First-class_Ave,BOWLING_First-class_Econ,BOWLING_First-class_SR,BOWLING_First-class_4w,BOWLING_First-class_5w,BOWLING_First-class_10,BOWLING_List A_Mat,BOWLING_List A_Inns,BOWLING_List A_Balls,BOWLING_List A_Runs,BOWLING_List A_Wkts,BOWLING_List A_BBI,BOWLING_List A_BBM,BOWLING_List A_Ave,BOWLING_List A_Econ,BOWLING_List A_SR,BOWLING_List A_4w,BOWLING_List A_5w,BOWLING_List A_10,BOWLING_T20s_Mat,BOWLING_T20s_Inns,BOWLING_T20s_Balls,BOWLING_T20s_Runs,BOWLING_T20s_Wkts,BOWLING_T20s_BBI,BOWLING_T20s_BBM,BOWLING_T20s_Ave,BOWLING_T20s_Econ,BOWLING_T20s_SR,BOWLING_T20s_4w,BOWLING_T20s_5w,BOWLING_T20s_10
0,8772,Henry Arkell,England,Henry John Denham Arkell,"\nJune 26, 1898, Edmonton, Middlesex","March 12, 1982, Oxford (aged 83 years 259 days)",NaN,Northamptonshire,NaN,NaN,NaN,NaN,Right-hand bat,NaN,NaN,NaN,NaN,[],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2,0,11,6,5.50,NaN,NaN,0,0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,-,-,-,-,-,-,-,-,-,-,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN